In [ ]:

import matplotlib
matplotlib.use("PDF")
import pyfits as pf
from pylab import *
import sys, glob
from scipy import stats

thedir='/Users/cslage/Research/LSST/optical_simulator/CTE/binning'
%cd $thedir

#%matplotlib inline

In [ ]:
files = sort(glob.glob('ITL-3800C-029_spot_spot_10??_20170228??????.fits'))
print files
name = files[0][24:28]
print name

In [ ]:
xstop = 128
ystop = 500
cti_list = []
cti_means= []
cti_stds = []
segments = [2,3,10,11]
num_segments = len(segments)
skys = [60.0, 400.0, 2400.0, 10000.0]
skip_segments = ["SEGMENT00","SEGMENT01","SEGMENT07","SEGMENT10","SEGMENT17"]
data_maxs=[22000.0, 22400.0, 24400.0, 32000.0]
fig = figure(figsize=(15,15))
subplots_adjust(hspace = 0.3, wspace = 0.3)
suptitle("Binned CTE(4x4) - EXPTIME = 4 seconds",fontsize=32)

for n,file in enumerate(files[0:4]):
    print file
    name = file[24:28]
    cti_ratios = []
    hdulist = pf.open(file, mode='readonly', do_not_scale_image_data=True)
    exptime = hdulist[0].header['EXPTIME']
    for p, segment in enumerate(segments):#range(16):
        nmax = 0
        cti_runmean = 0.0
        
        pointlist = []
        xaxis = linspace(-2,2,5)
        subplot(4,4,4*n+p+1)
        imhdr=hdulist[segment+1].header
        extname = imhdr['EXTNAME']
        title("%s - %.0f e- Sky"%(extname,skys[n]), fontsize=14)
        data = array(hdulist[segment+1].data + 32768, dtype = int32)[0:ystop,0:xstop]
        flat_overscan = median(data[0:ystop,0:xstop], axis = 1)
        cte_data = transpose(transpose(data) - flat_overscan)
        data_max = data_maxs[n]
        for i in range(2,xstop-2):
                for j in range(2,ystop-2):
                    # First keep only pixels with near the total amount of charge
                    if cte_data[j,i] < data_max * 0.7:
                        continue
                    sum_surround = float(sum(cte_data[j-1:j+2,i-1:i+2]) - cte_data[j,i])
                    sum_2x = float(cte_data[j,i-2] + cte_data[j,i+2])
                    # Then keep only pixels where the surroundings have less than 2% of the center pixel
                    if (sum_surround / cte_data[j,i]) > 0.02 or (sum_2x / cte_data[j,i]) > 0.02:
                        continue
                    pointlist.append([i,j]) # this is a list of "good pixel locations
                    nmax += 1
        spotdata = zeros([nmax,5])
        cti_ratio = zeros([nmax])
        print name, nmax
        for m,[i,j] in enumerate(pointlist):
            spotdata[m,:] = cte_data[j,i-2:i+3] - (amin(cte_data[j,i-2:i+3]) - 1)
            cti_ratio[m] = float(cte_data[j,i+1] - cte_data[j,i-1]) / float(cte_data[j,i])
            if extname not in skip_segments:
                cti_list.append([i*4,cti_ratio[m],segment,float(exptime)])
        cti_mean = mean(cti_ratio)
        cti_std = std(cti_ratio)
        for ratio in cti_ratio:
            cti_ratios.append(ratio)
        scatter(xaxis,mean(spotdata[:,:],axis=0))
        errorbar(xaxis, mean(spotdata[:,:],axis=0), yerr=std(spotdata[:,:],axis=0))
        plot([-2,2],[35000,35000], color = 'red', ls = '--')
        yscale('log')
        ylim(1.0,1.0E5)
        xlim(-2.2,2.2)
        text(-2.0,4.0E4,"CTI_Ratio = %.4f +/- %.4f"%(cti_mean,cti_std))
        print "%s, nmax = %d, cti_mean = %f"%(extname,nmax, cti_mean)
    cti_means.append(mean(cti_ratios))
    cti_stds.append(std(cti_ratios))

savefig("Serial_CTE_Binning_Sky_4us_Low_25Apr18.pdf")
close(fig)
print skys, cti_means, cti_stds
fig = figure(figsize=(15,15))
title("CTI Ratio vs Sky Level",fontsize=32)
errorbar(skys, cti_means, yerr=cti_stds/sqrt(len(cti_ratios)), marker = 'o')
#scatter(skys, cti_means)
xlabel("Sky Level (e-)", fontsize = 24)
ylabel("CTI Ratio", fontsize = 24)
xlim(-1000.0,12000.0)
ylim(0.0, 0.01)
xticks([0.0,2000.0, 4000.0, 6000.0, 8000.0, 10000.0], fontsize = 18)
yticks([0.00, 0.005, 0.01], fontsize = 18)
savefig("Serial_CTE_Binning_vs_Sky_25Apr18.pdf")
"""
figure()
title("CTI Ratios vs X-Position and Flux - 4x4")
cti1=[]
cti2=[]
cti3=[]
cti4=[]
cti5=[]

for [x,ratio,segment,exptime] in cti_list: 
    if exptime < 0.6:
        scatter([x],[ratio],s=0.5,color = 'red')
        cti1.append(ratio)
    elif exptime < 1.1:
        scatter([x+0.8],[ratio],s=0.5,color = 'blue')
        cti2.append(ratio)
    elif exptime < 1.6:
        scatter([x+1.6],[ratio],s=0.5,color = 'green')
        cti3.append(ratio)
    elif exptime < 2.1:
        scatter([x+2.4],[ratio],s=0.5,color = 'cyan')
        cti4.append(ratio)
    elif exptime < 2.6:
        scatter([x+3.2],[ratio],s=0.5,color = 'magenta')
        cti5.append(ratio)

text(50,0.0375,"Peak flux = 22Ke-, mean = %.4f, std = %.4f"%(mean(cti1),std(cti1)),color='red')
text(50,0.0350,"Peak flux = 44Ke-, mean = %.4f, std = %.4f"%(mean(cti2),std(cti2)),color='blue')
text(50,0.0325,"Peak flux = 66Ke-, mean = %.4f, std = %.4f"%(mean(cti3),std(cti3)),color='green')
text(50,0.0300,"Peak flux = 88Ke-, mean = %.4f, std = %.4f"%(mean(cti4),std(cti4)),color='cyan')
text(50,0.0275,"Peak flux = 110Ke-, mean = %.4f, std = %.4f"%(mean(cti5),std(cti5)),color='magenta')
plot([0,520],[0.0,0.0], ls = '--', color = 'black')
ylim(-0.01, 0.04)
xlim(0,520)
xlabel("X Location")
ylabel("CTI Ratio")
savefig("Serial_CTE_Binning_Ratios_4x4_27Feb17.pdf")
"""
